# Extracting from TMDB
Brittany Lassiter

## Imports

In [1]:
# Import packages
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

In [2]:
# making new folder with os
import os
os.makedirs('Data/',exist_ok=True)

In [3]:
# Confirm folder was created and files added successfully
os.listdir("Data/")

['.ipynb_checkpoints',
 'IMDB Movie Dataset Info (2).docx',
 'movies-erd.png',
 'movies.2.mwb',
 'Movies.sql',
 'title-akas-us-only.xls',
 'title.basics (3).tsv.gz',
 'title.ratings (2).tsv.gz',
 'title_basics_cleaned.csv.gz',
 'title_ratings_cleaned.csv.gz']

In [4]:
!pip install tqdm
!pip install tmdbsimple

In [5]:
import json
with open("/Users/blass/.secret/tmdb_api.json", 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['API Key'])

In [6]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['API Key']

## Define Search

In [26]:
# set our API call parameters 
Year = '2001 or 2002'

## Create JSON File

In [27]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/title_basics_cleaned.csv.gz"
JSON_FILE

'Data/title_basics_cleaned.csv.gz'

In [28]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/title_basics_cleaned.csv.gz already exists.
